# Ostrofsky's rootN times rootN PIR scheme based on Learning with Errors and Regev Encryption :

Using parameters from the paper: **One server for the price of Two: Simple and Fast Single-Server Private Information Retreival**

Here the number of LWE samples(m) = m

In [378]:
import numpy as np


# parameters
n = 2**10
m = 2 # sqrt(N), number of samples

error_values = {
    "loc": 0,
    "sigma": 6.4,
}
q = 2**23
p = 991 # plaintext modulus
scaling_factor = np.floor(q/p)

x = np.array([0, 1])
print(f"scaling_factor = floor(q/p) = {np.floor(q/p)}")

print(x * scaling_factor)
print(f"x = {x}")

db = np.random.randint(0, p, (m,m))
print(f"db = {db}")

scaling_factor = floor(q/p) = 8464.0
[   0. 8464.]
x = [0 1]
db = [[413 837]
 [372 756]]


In [379]:
#using matrices (when you have m samples)


def sampleLWE(m,n,q,error_values, s):
    A= np.random.randint(0, q, (m,n))
    
    e= np.random.normal(error_values["loc"], error_values["sigma"], (m) )

    print(f"A = {A}")
    print(f"s = {s}")
    # print(f"e = {e}")


    # Computing the dot product and adding the error

    # Outputting the vectors a, s and the value b

    b = (np.dot(A,s)  + e ) % q
    print(f"b = As + e mod q = {b}")
    b_prime = (np.dot(A,s) % q + e) % q


    return A, b




In [380]:
# print(f"A = ", A)
# print(type(A), A.shape)
# print(f"s = ", s)
# print(type(s), s.shape)
# print(f"e = ", e)
# print(type(e), e.shape)

In [381]:
secret= np.random.randint(0, q, (n))

A, b = sampleLWE(m,n,q,error_values, secret)

A = [[7621029 6116754 2994212 ... 4374130 3394543 5963646]
 [8027628 7927635 5821658 ... 6863943 3347644 1839983]]
s = [8241637 5237951  502421 ... 1450150  759611  330145]
b = As + e mod q = [1705812. 1452036.]


In [382]:
# print(f"as mod q = {np.dot(A, s)% q}")

# print(f"(b = As mod q + e ) mod q = ", b)

# print(f" b_prime = As + e mod q    = {b_prime}")

In [383]:
# Encryption

def regevEncrypt(b, x, p, q):
    scaling_factor = np.floor(q/p)
    c =( b + scaling_factor*x ) 
    c_mod_q = c % q

    

    # print(f"c= As + e + (floor(q/p) * x) \n \\ \
    # = As + e + {scaling_factor}*{x} = {c} = {c_mod_q}")

    return c_mod_q

In [384]:


c = regevEncrypt(b, x, p, q)
print(c)

[1705812. 1460500.]


Send Client --- (mu_enc = A, c) ----> Server

In [385]:
# server computations:

# D dot A

DA = np.dot(db,A) % q
Dc = np.dot(db,c) % q

print(DA)
print(Dc)

[[1606605 1289961 2430678 ... 1888781 1231679 1685553]
 [3596468 5945668 3704856 ... 4767572 1938044 2402020]]
[5947624. 2258208.]


In [386]:
#Decryption

# print(np.dot(a, s), np.dot(a,s) % q)

def rounding(x, round_off_to):
    """
    Rounds off a number x to the nearest multiple of round_off_to.

    :param x: The number to be rounded off.
    :param round_off_to: The nearest multiple to which x needs to be rounded off.
    :return: The rounded off value.
    """
    return np.round(x / round_off_to) * round_off_to

def regevDecrpt(A, s, q, p, c):
    scaling_factor = np.floor(q/p)
    # a_s_mod_q = np.dot(A,s)%q 
    # print(f"a_s_modq = as mod q = ", a_s_mod_q)

    d_hat = (c - np.dot(A, s)) % q 

    print(f"d_hat = c - as mod q= {c} - {np.dot(A,s)} mod q = ", d_hat)
    rounded_d = rounding(d_hat, scaling_factor)



    print(f"rounded d = {rounded_d}")
    
    d = rounded_d / scaling_factor
    print(f"d = rounded_d/ floor(q/p)= {rounded_d}/ {scaling_factor} = {d}" )
    return d






In [387]:
print(DA)
print(Dc)

[[1606605 1289961 2430678 ... 1888781 1231679 1685553]
 [3596468 5945668 3704856 ... 4767572 1938044 2402020]]
[5947624. 2258208.]


In [390]:
d_hat = ( Dc[0] - np.dot(DA[0], secret) ) % q
print(f"d_hat = {d_hat}")

d_hat_prime = ( Dc[1] - (np.dot(DA[1], secret)) %q ) % q
print(f"d_hat_prime = {d_hat_prime}")

rounded_d = rounding(d_hat_prime, scaling_factor)



print(f"rounded d = {rounded_d}")

d = rounded_d / scaling_factor
print(f"d = rounded_d/ floor(q/p)= {rounded_d}/ {scaling_factor} = {d}" )

print(db)

d_hat = 7081500.0
d_hat_prime = 6396204.0
rounded d = 6398784.0
d = rounded_d/ floor(q/p)= 6398784.0/ 8464.0 = 756.0
[[413 837]
 [372 756]]


In [389]:
print(f"x = {x}")
d = regevDecrpt(A, secret, q, p, c = Dc[0])

x = [0 1]
d_hat = c - as mod q= 5947624.0 - [18490217150941023 18518784018360322] mod q =  [4241800. 4495592.]
rounded d = [4240464. 4494384.]
d = rounded_d/ floor(q/p)= [4240464. 4494384.]/ 8464.0 = [501. 531.]
